In [18]:
import pandas as pd
import pandas_ta as ta
import numpy as np 
import random

In [19]:
random.seed(42)

In [25]:
date_idx  = pd.DatetimeIndex(pd.date_range('20230101','20230110'))
rec_count = len(date_idx)
df = pd.DataFrame({'Date':date_idx, 'Close': [random.randint(10,100) for _ in range(rec_count)]})

In [28]:
df['Ret']=df.Close.pct_change()

In [29]:
df['Pred'] = [random.random() for _ in range(rec_count)]

In [30]:
df["Ret25_i"] = df[name].rolling(25).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))
df["Ret60_i"] = df[name].rolling(60).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))
df["Ret90_i"] = df[name].rolling(90).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))
df["Ret120_i"] = df[name].rolling(120).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))
df["Ret240_i"] = df[name].rolling(240).apply(lambda x: 100 * (np.prod(1 + x / 100) - 1))


df["Ret120"] = df["Ret120_i"].shift(-120)
df["Output"] = df["Ret120"] > 0
df["Output"] = df["Output"].astype(int)
del df["Ret120"]
df = df.dropna()
df.tail(10)

,Date,Close,Ret,Pred
0,2023-01-01,85,NaN,0.957213
1,2023-01-02,45,-0.470588,0.336595
2,2023-01-03,10,-0.777778,0.092746
3,2023-01-04,30,2.000000,0.096716
4,2023-01-05,99,2.300000,0.847494
5,2023-01-06,64,-0.353535,0.603726
6,2023-01-07,53,-0.171875,0.807128
7,2023-01-08,45,-0.150943,0.729732
8,2023-01-09,29,-0.355556,0.536228
9,2023-01-10,37,0.275862,0.973116


In [31]:
df["Positions"] = np.where(df["Pred"] > 0.5, 1, -1)
df["Strat_ret"] = df["Positions"].shift(1) * df["Ret"]
df["Positions_L"] = df["Positions"].shift(1)
df["Positions_L"][df["Positions_L"] == -1] = 0
df["Strat_ret_L"] = df["Positions_L"] * df["Ret"]
df["CumRet"] = df["Strat_ret"].expanding().apply(lambda x: np.prod(1 + x) - 1)
df["CumRet_L"] = df["Strat_ret_L"].expanding().apply(lambda x: np.prod(1 + x) - 1)
df["bhRet"] = df["Ret"].expanding().apply(lambda x: np.prod(1 + x) - 1)

Final_Return_L = np.prod(1 + df["Strat_ret_L"]) - 1
Final_Return = np.prod(1 + df["Strat_ret"]) - 1
Buy_Return = np.prod(1 + df["Ret"]) - 1

print("Strat Return Long Only =", Final_Return_L * 100, "%")
print("Strat Return =", Final_Return * 100, "%")
print("Buy and Hold Return =", Buy_Return * 100, "%")

Strat Return Long Only = -80.21390374331551 %
Strat Return = -54.2721330956625 %
Buy and Hold Return = -56.470588235294116 %


/var/folders/mk/pkqnv_dx053g95zwc3mj01b00000gn/T/ipykernel_21795/110887527.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Positions_L"][df["Positions_L"] == -1] = 0
